In [ ]:
from cudf import DataFrame
import cudf
import math
from math import log
import nvstrings, nvcategory
from numba import cuda, float32
import time
import rmm 
import numpy as np
from cudf.utils import cudautils

### Q5. Series UDFs: example using forall API 
(reference link: https://docs.rapids.ai/api/cudf/stable/guide-to-udfs.html#Numba-Kernels-on-CuPy-Arrays#:~:text=The%20easiest%20way%20to,on%20our%20Series)

In [ ]:
@cuda.jit
def add(in_col, out_col, num):
    i = cuda.grid(1)
    if i < in_col.size: # boundary guard
        out_col[i] = in_col[i] + num

In [ ]:
n = 1000 # data size
a = cudf.Series(cudautils.zeros(n, dtype='float32'))
e = cudf.Series(cudautils.zeros(n, dtype='float32'))
add.forall(n)(a, e, 10.0)

### Q6. DataFrame UDFs: example using forall
(reference link: https://docs.rapids.ai/api/cudf/stable/guide-to-udfs.html#Numba-Kernels-on-CuPy-Arrays#:~:text=The%20easiest%20way%20to,on%20our%20Series)

In [ ]:
@cuda.jit
def multiply_1(in_col, out_col, multiplier):
    i = cuda.grid(1)
    if i < in_col.size: # boundary guard
        out_col[i] = in_col[i] * multiplier

In [ ]:
df = cudf.DataFrame({'a':[1,2]*50})
df['e'] = 0
size = df.shape[0]
multiply_1.forall(size)(df['a'], df['e'], 10.0)

### Q7. DataFrame UDFs: example using apply_rows
(reference link: https://docs.rapids.ai/api/cudf/stable/guide-to-udfs.html#Numba-Kernels-on-CuPy-Arrays#:~:text=We%20could%20apply%20a%20UDF,on%20the%20core%20data%20structures%20of%20cuDF

In [ ]:
def multiply_2(x, e, multiplier):
    for i, a in enumerate(x):
        e[i] = a * multiplier
        

In [ ]:
df = cudf.DataFrame({'a':[1,2]*50})
df = df.apply_rows(multiply_2,
                   incols={'a':'x'},
                   outcols={'e': np.float64},
                   kwargs={'multiplier':3}
                  )